In [1]:
import joblib
import ncnn
import numpy as np

from sklearn.datasets import fetch_openml

from skl2onnx import to_onnx
from skl_mlp2ncnn import convert

In [2]:
# load data
X, y = fetch_openml(
    name="mnist_784",
    version=1,
    return_X_y=True,
    as_frame=False,
    parser="pandas",
)

X = X / 255.0

In [3]:
# load model and infer with scikit-learn
classifier = joblib.load("mnist_classifier.pkl")
label_skl = classifier.predict(X[:1])
proba_skl = classifier.predict_proba(X[:1])

In [4]:
# convert to onnx
onx = to_onnx(classifier, X[:1])

with open("mnist_classifier.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [5]:
# convert
ncnn_param, ncnn_bin = convert(
    skl_mlp=classifier,
    ncnn="mnist_classifier",
    class_table="mnist.txt",
    verbose=True,
)

Input model is MLPClassfier model.
classes will be saved to file mnist.txt.
                                      INFO                                      
--------------------------------------------------------------------------------
    Layer 1: Input
        Input shape: (784,)
--------------------------------------------------------------------------------
    Layer 2: InnerProduct
        Output shape: (512,)
--------------------------------------------------------------------------------
    Layer 3: ReLU
--------------------------------------------------------------------------------
    Layer 4: InnerProduct
        Output shape: (512,)
--------------------------------------------------------------------------------
    Layer 5: ReLU
--------------------------------------------------------------------------------
    Layer 6: InnerProduct
        Output shape: (10,)
--------------------------------------------------------------------------------
    Layer 7: Softmax
--------

In [6]:
print(f"param file: {ncnn_param}")
print(f"bin file: {ncnn_bin}")

param file: mnist_classifier.param
bin file: mnist_classifier.bin


In [7]:
# load ncnn model
with ncnn.Net() as net:
    net.load_param(ncnn_param)
    net.load_model(ncnn_bin)

    with net.create_extractor() as ex:
        ex.input("input_blob", ncnn.Mat(X[0].astype(np.float32)))
        _, out = ex.extract("layer_2_softmax_blob")

proba_ncnn = np.array(out)

class_table = np.loadtxt("mnist.txt", dtype="<U1").reshape((-1, 1))
label_ncnn = class_table[np.argmax(proba_ncnn)]

In [8]:
# summary
print("scikit-learn:")
print(f"    label: {label_skl}")
print(f"    probability: {proba_skl}")

print("ncnn:")
print(f"    label: {label_ncnn}")
print(f"    probability: {proba_ncnn}")

scikit-learn:
    label: ['5']
    probability: [[2.17463843e-15 3.10142755e-12 6.31065117e-11 3.05429930e-04
  1.52861992e-22 9.99694570e-01 1.36513153e-15 5.91957558e-15
  1.02861526e-14 1.24418613e-12]]
ncnn:
    label: ['5']
    probability: [2.22792789e-15 3.16377133e-12 6.41421222e-11 3.06921516e-04
 1.58592659e-22 9.99693155e-01 1.39784958e-15 6.05439754e-15
 1.05316765e-14 1.26826490e-12]
